In [2]:
arcpy.env.overwriteOutput = True

In [12]:
# Alex Chansky
# 3/8/21, 3/9/21, 3/10/21
# Purpose: Creating polygons of 1 km buffers and 15 km buffers away from points
            # and polygons. Need to set up three separate functions to run depending on
            # whether an event has only liq points, only liq polygons, or both.
import arcpy
from arcpy import env
import pandas as pd
import os
arcpy.CheckOutExtension("Spatial") # This checks out the license
from arcpy.sa import *
# If getting "Not Signed into Portal" message, should open up and log into ArcGIS Pro

In [3]:
path = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\"

In [4]:
temp_path = "C:\\Users\\achans01\\Downloads\\Arc"

In [13]:
eq_list = pd.read_csv("R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\z_eq_names\\2021_eq_names.csv")
num_eq = eq_list.shape[0]
eq_list_df = pd.DataFrame(eq_list)

In [6]:
# PTS ONLY GEODESIC
# Function for 1 km and 15 km buffers on Point geometries
def ptsonly(eq_name):
    points = path + eq_name + "\\Liq_point.shp"

    # Removing specification of points or polys so that all buffers can be viewed as equivalent
    one_km_buff = path + eq_name + "\\1km_buffer.shp"
    fifteen_km_buff = path + eq_name + "\\15km_buffer.shp"
    
    arcpy.Buffer_analysis(points, one_km_buff, "1000 meters", "", "", "ALL","","GEODESIC")
    arcpy.Buffer_analysis(points, fifteen_km_buff, "15000 meters", "", "", "ALL","","GEODESIC")

In [7]:
# POLY ONLY GEODESIC
# Function for 1 km and 15 km buffers on Poly geometries
def polyonly(eq_name):
    poly = path + eq_name + "\\Liq_poly.shp"
    
    # Removing specification of points or polys so that all buffers can be viewed as equivalent
    one_km_buff = path + eq_name + "\\1km_buffer.shp"
    fifteen_km_buff = path + eq_name + "\\15km_buffer.shp"
    print("a")
    
    # Building wide buffer around small buffer is MUCH quicker than building wide buffer from polygons, even if merged
    arcpy.Buffer_analysis(poly, one_km_buff, "1000 meters", "", "", "ALL","","GEODESIC")
    print("b")
    # For some polygon layers, Arc is struggling to produce this larger buffer. So we will
    # use the smaller buffer and create a 14km buffer on top of it
    arcpy.Buffer_analysis(one_km_buff, fifteen_km_buff, "14000 meters", "", "", "ALL","","GEODESIC")
    print("c")

In [8]:
# BOTH GEODESIC
# Function for 1 km and 15 km buffers on Both geometries
def both(eq_name):
    points = path + eq_name + "\\Liq_point.shp"
    poly = path + eq_name + "\\Liq_poly.shp"
    print("a")
    
    one_km_buff_pts = temp_path + "\\1km_buffer_pts.shp"
    arcpy.Buffer_analysis(points, one_km_buff_pts, "1000 meters", "", "", "ALL","","GEODESIC")
    print("b")
    
    one_km_buff_poly = temp_path + "\\1km_buffer_poly.shp"
    arcpy.Buffer_analysis(poly, one_km_buff_poly, "1000 meters", "", "", "ALL","","GEODESIC")
    print("c")
    
    one_km_buff = path + eq_name + "\\1km_buffer.shp"
    fifteen_km_buff = path + eq_name + "\\15km_buffer.shp"
    print("d")
    
    arcpy.management.Merge([one_km_buff_pts, one_km_buff_poly], one_km_buff)
    arcpy.Buffer_analysis(one_km_buff, fifteen_km_buff, "14000 meters", "", "", "ALL","","GEODESIC")

In [10]:
# Some reason overwrite is not working, so these are necessary to ensure everything in temp_path is deleted
def deletey(): 
    one_km_buff_pts = temp_path + "\\1km_buffer_pts.shp"
    fifteen_km_buff_pts = temp_path + "\\15km_buffer_pts.shp"
    one_km_buff_poly = temp_path + "\\1km_buffer_poly.shp"
    fifteen_km_buff_poly = temp_path + "\\15km_buffer_poly.shp"

    arcpy.Delete_management(one_km_buff_pts)
    arcpy.Delete_management(fifteen_km_buff_pts)
    arcpy.Delete_management(one_km_buff_poly)
    arcpy.Delete_management(fifteen_km_buff_poly)

In [14]:
for x in range(35,36):
    eq_name = eq_list['NewFolder'][x]
    if eq_list['Geometry'][x] == "PtsOnly" and eq_list['Working'][x]==1:
        ptsonly(eq_name)
    elif eq_list['Geometry'][x] == "PolyOnly" and eq_list['Working'][x]==1:
        polyonly(eq_name)
    elif eq_list['Geometry'][x] == "Both" and eq_list['Working'][x]==1:
        both(eq_name)
    else:
        pass
    
    
    # Some reason overwrite is not working, so these are necessary to ensure everything in temp_path is deleted
    deletey()
    print("Successfully completed " + str(x) + ", " + eq_name)
    
    # Ignore long bug stating "<built-in function isinstance> returned a result with an error set"
    # Output of this will be adequate buffers for all events  

a
b
c
d
Successfully completed 35, Nihonkai
